Pandas features a number of functions for reading tabular data as a DataFrame
object, though read_csv and read_table are
likely the ones you’ll use the most.

I’ll give an overview of the mechanics of these functions, which are meant to convert
text data into a DataFrame. The optional arguments for these functions may fall into
a few categories:

**Indexing:** Can treat one or more columns as the returned DataFrame, and whether to get
column names from the file, the user, or not at all.

**Type inference and data conversion:**
This includes the user-defined value conversions and custom list of missing value
markers.

**Datetime parsing:**
Includes combining capability, including combining date and time information
spread over multiple columns into a single column in the result.

**Iterating:**
Support for iterating over chunks of very large files.

**Unclean data issues:**
Skipping rows or a footer, comments, or other minor things like numeric data
with thousands separated by commas.

Because of how messy data in the real world can be, some of the data loading func‐
tions (especially read_csv) have grown very complex in their options over time. It’s
normal to feel overwhelmed by the number of different parameters (read_csv has
over 50 as of this writing). The online pandas documentation has many examples
about how each of them works, so if you’re struggling to read a particular file, there
might be a similar enough example to help you find the right parameters.

Some of these functions, like pandas.read_csv, perform type inference, because the
column data types are not part of the data format. That means you don’t necessarily
have to specify which columns are numeric, integer, boolean, or string. Other data
formats, like HDF5, Feather, and msgpack, have the data types stored in the format.

Handling dates and other custom types can require extra effort. Let’s start with a
small comma-separated (CSV) text file:

In [1]:
import pandas as pd
import numpy as np
import csv
import json
import sqlite3

df = pd.read_csv('./pydata-book/examples/ex1.csv')
df

We could also have used read_table and specified the delimiter

In [2]:
pd.read_table('./pydata-book/examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


A file will not always have a header row. Consider this file:

In [3]:
pd.read_csv('./pydata-book/examples/ex2.csv', header=None)             

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


To read this file, you have a couple of options. You can allow pandas to assign default
column names, or you can specify names yourself:

In [4]:
pd.read_csv('./pydata-book/examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Suppose you wanted the message column to be the index of the returned DataFrame.
You can either indicate you want the column at index 4 or named 'message' using
the index_col argument:

In [5]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('./pydata-book/examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In the event that you want to form a hierarchical index from multiple columns, pass a
list of column numbers or names:

In [6]:
parsad = pd.read_csv('./examples/csv_mindex.csv')
parsad

,key1,key2,value1,value2
0,one,a,1,2
1,one,b,3,4
2,one,c,5,6
3,one,d,7,8
4,two,a,9,10
5,two,b,11,12
6,two,c,13,14
7,two,d,15,16


In [7]:
pd.read_csv('./examples/csv_mindex.csv', index_col=['key1', 'key2'])

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In some cases, a table might not have a fixed delimiter, using whitespace or some
other pattern to separate fields. Consider a text file that looks like this:

In [8]:
list(open('./examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

While you could do some munging by hand, the fields here are separated by a vari‐
able amount of whitespace. In these cases, you can pass a regular expression as a
delimiter for read_table. This can be expressed by the regular expression \s+, so we
have then:

In [9]:
pd.read_table('./examples/ex3.txt', sep='\s+')

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


Because there was one fewer column name than the number of data rows,
read_table infers that the first column should be the DataFrame’s index in this spe‐
cial case.

The parser functions have many additional arguments to help you handle the wide
variety of exception file formats that occur (see a partial listing in Table 6-2). For
example, you can skip the first, third, and fourth rows of a file with skiprows:

In [10]:
pd.read_csv('./examples/ex4.csv')

,,,,# hey!
a,b,c,d,message
# just wanted to make things more difficult for you,NaN,NaN,NaN,NaN
# who reads CSV files with computers,anyway?,NaN,NaN,NaN
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [11]:
pd.read_csv('./examples/ex4.csv', skiprows=[0, 2, 3, 4])

,a,b,c,d,message
0,5,6,7,8,world
1,9,10,11,12,foo


Handling missing values is an important and frequently nuanced part of the file pars‐
ing process. Missing data is usually either not present (empty string) or marked by
some sentinel value. By default, pandas uses a set of commonly occurring sentinels,
such as NA and NULL:

In [12]:
results = pd.read_csv('examples/ex5.csv')
results

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [13]:
pd.isnull(results)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


The na_values option can take either a list or set of strings to consider missing
values:

In [14]:
results = pd.read_csv('examples/ex5.csv', na_values=['NULL'])
results

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


## Reading Text Files in Pieces

When processing very large files or figuring out the right set of arguments to cor‐
rectly process a large file, you may only want to read in a small piece of a file or iterate
through smaller chunks of the file.

Before we look at a large file, we make the pandas display settings more compact:


In [15]:
pd.options.display.max_rows = 10

Now we have:

In [16]:
result = pd.read_csv('./examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


If you want to only read a small number of rows (avoiding reading the entire file),
specify that with nrows:

In [17]:
result = pd.read_csv('./examples/ex6.csv', nrows=10)
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
5,1.817480,0.742273,0.419395,-2.251035,Q
6,-0.776764,0.935518,-0.332872,-1.875641,U
7,-0.913135,1.530624,-0.572657,0.477252,K
8,0.358480,-0.497572,-0.367016,0.507702,S
9,-1.740877,-1.160417,-1.637830,2.172201,G


To read a file in pieces, specify a chunksize as a number of rows:


In [18]:
chunker = pd.read_csv('./examples/ex6.csv', chunksize=1000)
chunker

The TextParser object returned by read_csv allows you to iterate over the parts of
the file according to the chunksize. For example, we can iterate over ex6.csv, aggre‐
gating the value counts in the 'key' column like so:

In [19]:
tot = pd.Series([], dtype='float64')


In [20]:
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

TextParser is also equipped with a get_chunk method that enables you to read
pieces of an arbitrary size

## Writing Data to Text Format


Data can also be exported to a delimited format. Let’s consider one of the CSV files
read before:    

In [21]:
data = pd.read_csv('./examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Using DataFrame’s to_csv method, we can write the data out to a comma-separated
file:

In [22]:
data.to_csv('ex5.csv', index=False, header=False)

In [23]:
pd.read_csv('ex5.csv',)

,one,1,2,3.0,4,Unnamed: 5
0,two,5,6,NaN,8,world
1,three,9,10,11.0,12,foo


You can also write only a subset of the columns, and in an order of your choosing:


In [24]:
data.to_csv('ex5.csv', index=False, columns=['a', 'b', 'c'])

In [25]:
pd.read_csv('ex5.csv',)

,a,b,c
0,1,2,3.0
1,5,6,NaN
2,9,10,11.0


Series also has a to_csv method:


In [26]:
dates = pd.date_range('1/1/2000', periods=7)
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07'],
              dtype='datetime64[ns]', freq='D')

In [27]:
ts = pd.Series(np.arange(7), index=dates)
ts

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int32

In [28]:
ts.to_csv('tseries.csv', header=False)

In [29]:
pd.read_csv('tseries.csv', header=None)

,0,1
0,2000-01-01,0
1,2000-01-02,1
2,2000-01-03,2
3,2000-01-04,3
4,2000-01-05,4
5,2000-01-06,5
6,2000-01-07,6


## Working with Delimited Formats


It’s possible to load most forms of tabular data from disk using functions like pan
das.read_table. In some cases, however, some manual processing may be necessary.
It’s not uncommon to receive a file with one or more malformed lines that trip up
read_table. To illustrate the basic tools, consider a small CSV file:

In [30]:
pd.read_csv('./examples/ex7.csv', header=None)

,0,1,2
0,a,b,c
1,1,2,3
2,1,2,3


For any file with a single-character delimiter, you can use Python’s built-in csv mod‐
ule. To use it, pass any open file or file-like object to csv.reader:

In [31]:
f = open('./examples/ex7.csv')
reader = csv.reader(f)

Iterating through the reader like a file yields tuples of values with any quote charac‐
ters removed:

In [32]:
reader

In [33]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


From there, it’s up to you to do the wrangling necessary to put the data in the form
that you need it. Let’s take this step by step. First, we read the file into a list of lines:

In [34]:
with open('./examples/ex7.csv') as file:
    lines = list(csv.reader(file))
lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3']]

Then, we split the lines into the header line and the data lines:


In [35]:
header, values = lines[0], lines[1:]
header, values

(['a', 'b', 'c'], [['1', '2', '3'], ['1', '2', '3']])

Then we can create a dictionary of data columns using a dictionary comprehension
and the expression zip(*values), which transposes rows to columns:

In [36]:
data_dict = dict(list(zip(header, zip(*values))))
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [37]:
data_dict = { h:v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

To write delimited files manually, you can use csv.writer. It accepts an open, writa‐
ble file object and the same dialect and format options as csv.reader

## JSON - (JavaScript Object Notation)

JSON has become one of the standard formats
for sending data by HTTP request between web browsers and other applications. It is
a much more free-form data format than a tabular text form like CSV. Here is an
example:

In [38]:
obj = """
{"name": "Wes",
"places_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
{"name": "Katie", "age": 38,
"pets": ["Sixes", "Stache", "Cisco"]}]
}
"""
obj

'\n{"name": "Wes",\n"places_lived": ["United States", "Spain", "Germany"],\n"pet": null,\n"siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},\n{"name": "Katie", "age": 38,\n"pets": ["Sixes", "Stache", "Cisco"]}]\n}\n'

JSON is very nearly valid Python code with the exception of its null value null and
some other nuances (such as disallowing trailing commas at the end of lists). The
basic types are objects (dicts), arrays (lists), strings, numbers, booleans, and nulls. All
of the keys in an object must be strings. There are several Python libraries for reading 
and writing JSON data. I’ll use json here, as it is built into the Python standard
library. To convert a JSON string to Py

In [39]:
results = json.loads(obj)
results

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

json.dumps, on the other hand, converts a Python object back to JSON:


In [40]:
asjson = json.dumps(results)
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

How you convert a JSON object or list of objects to a DataFrame or some other data
structure for analysis will be up to you. Conveniently, you can pass a list of dicts
(which were previously JSON objects) to the DataFrame constructor and select a sub‐
set of the data fields:

In [41]:
list(results.keys())[1]

'places_lived'

In [42]:
pd.DataFrame(results['siblings'])

,name,age,pets
0,Scott,30,"[Zeus, Zuko]"
1,Katie,38,"[Sixes, Stache, Cisco]"


The default options for pandas.read_json assume that each object in the JSON array
is a row in the table:

In [43]:
data = pd.read_json('./examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


If you need to export data from pandas to JSON, one way is to use the to_json meth‐
ods on Series and DataFrame:

In [44]:
data.to_json()

'{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}'

In [45]:
data.to_json(orient=('records'))

'[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]'

## Binary Data Formats

One of the easiest ways to store data (also known as serialization) efficiently in binary
format is using Python’s built-in pickle serialization. pandas objects all have a
to_pickle method that writes the data to disk in pickle format:

In [46]:
frame = pd.read_csv('./examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [47]:
frame.to_pickle('./frame_pickle')

You can read any “pickled” object stored in a file by using the built-in pickle directly,
or even more conveniently using pandas.read_pickle:

In [48]:
pd.read_pickle('./frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo
